In [1]:
import nest_asyncio

from llama_index.embeddings import HuggingFaceEmbedding

nest_asyncio.apply()

embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")

f:\dev\simple-rag\.env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [37]:
from llama_index.llms import LlamaCPP

# model_url = "https://huggingface.co/TheBloke/zephyr-7B-beta-GGUF/resolve/main/zephyr-7b-beta.Q4_K_M.gguf"

llm = LlamaCPP(
    # model_url="https://huggingface.co/TheBloke/zephyr-7B-beta-GGUF/resolve/main/zephyr-7b-beta.Q4_K_M.gguf",
    # optionally, you can set the path to a pre-downloaded model instead of model_url
    model_path="./../llm/stablelm-zephyr-3b.Q4_K_M.gguf",
    temperature=0.1,
    max_new_tokens=256,
    # llama2 has a context window of 4096 tokens, but we set it lower to allow for some wiggle room
    context_window=3900,
    # kwargs to pass to __call__()
    generate_kwargs={},
    # kwargs to pass to __init__()
    # set to at least 1 to use GPU
    model_kwargs={"n_gpu_layers": -1},
    verbose=True,
)

AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 0 | SSE3 = 1 | SSSE3 = 0 | VSX = 0 | 


In [38]:
from llama_index import ServiceContext
from llama_index.callbacks import CallbackManager, LlamaDebugHandler

llama_debug = LlamaDebugHandler(print_trace_on_end=True)
callback_manager = CallbackManager([llama_debug])
service_context = ServiceContext.from_defaults(
    llm=llm, 
    embed_model=embed_model, 
    callback_manager=callback_manager,
)

In [39]:
import re
import glob
import chromadb

from llama_index import SimpleDirectoryReader, VectorStoreIndex
from llama_index.tools import QueryEngineTool, ToolMetadata
from llama_index.vector_stores import ChromaVectorStore
from llama_index.storage.storage_context import StorageContext

def parse_string(input_string):
    parsed_string = input_string.replace(' ', '_')
    parsed_string = parsed_string.replace('-', '_')
    parsed_string = re.sub(r'[^a-z0-9_]', '', parsed_string.lower())
    parsed_string = re.sub(r'(_)\1+', r'\1', parsed_string)
    
    return parsed_string

input_dir = "./../ainu_papers/"
supported_files = [".pdf", ".csv", ".docx", ".txt", ".epub", ".hwp", ".mbox", ".ppt", ".pptm", ".pptx", ".ipynb", ".md"]
files = [f for f in glob.glob(f"{input_dir}**/*", recursive=True) if f".{f.split('.')[-1]}" in supported_files]

db = chromadb.PersistentClient(path="./db")
query_engine_tools = []

for file in files:
    new = False
    title = " ".join(file.replace("\\", "/").split("/")[-1].split(".")[:-1])
    collection_name = parse_string(title)[:63]
    print(f"Loading file titled: {title}")
    try:
        chroma_collection = db.get_collection(collection_name)
    except:
        new = True
        documents = SimpleDirectoryReader(input_files=[file]).load_data()
        chroma_collection = db.create_collection(collection_name)

    print(f"Storing and Indexing file title: {title}")
    vector_store = ChromaVectorStore(chroma_collection=chroma_collection)

    if new:
        storage_context = StorageContext.from_defaults(vector_store=vector_store)
        index = VectorStoreIndex.from_documents(
            documents, 
            storage_context=storage_context, 
            service_context=service_context,
            use_async=True,
        )
    else:
        index = VectorStoreIndex.from_vector_store(
        vector_store,
        service_context=service_context,
        use_async=True,
    )
        
    print(f"Init query engine and add to qe_tools")
    # Query Data from the persisted index
    query_engine = index.as_query_engine()
    query_engine_tools.append(
        QueryEngineTool(
            query_engine=query_engine,
            metadata=ToolMetadata(
                name=collection_name,
                description=f"A Paper titled: {title}",
            )
        )
    )

Loading file titled: Grain size strengthening in terms of dislocation density measured by resistivity
Storing and Indexing file title: Grain size strengthening in terms of dislocation density measured by resistivity
**********
Trace: index_construction
**********
Init query engine and add to qe_tools
Loading file titled: [Armstrong] 60 Years of Hall-Petch - Past to Present Nano-Scale Connections
Storing and Indexing file title: [Armstrong] 60 Years of Hall-Petch - Past to Present Nano-Scale Connections
**********
Trace: index_construction
**********
Init query engine and add to qe_tools


In [40]:
from llama_index.query_engine import SubQuestionQueryEngine

subq_engine = SubQuestionQueryEngine.from_defaults(
    query_engine_tools=query_engine_tools,
    service_context=service_context,
    use_async=True
)

In [41]:
response = subq_engine.query("Why Nickel is chosen as material for the experimentation?")
print(response)

**********
Trace: query
    |_CBEventType.QUERY ->  15.542579 seconds
      |_CBEventType.TEMPLATING ->  0.0 seconds
      |_CBEventType.LLM ->  15.542579 seconds
**********


ValidationError: 1 validation error for SubQuestion
tool_name
  none is not an allowed value (type=type_error.none.not_allowed)

In [ ]:
# iterate through sub_question items captured in SUB_QUESTION event
from llama_index.callbacks.schema import CBEventType, EventPayload

for i, (start_event, end_event) in enumerate(
    llama_debug.get_event_pairs(CBEventType.SUB_QUESTION)
):
    qa_pair = end_event.payload[EventPayload.SUB_QUESTION]
    print("Sub Question " + str(i) + ": " + qa_pair.sub_q.sub_question.strip())
    print("Answer: " + qa_pair.answer.strip())
    print("====================================")

Sub Question 0: What are the advantages of using nickel in experimental research?
Answer: Nickel has several advantages when used in experimental research, including:
1. High stacking fault energy: Nickel has a high stacking fault energy, which makes it suitable for studying dislocation density and its effect on flow stress.
2. Easy to work with: Nickel is a relatively easy material to work with, as it can be easily melted and shaped into various forms.
3. High electrical resistivity: Nickel has a high electrical resistivity, which allows for the measurement of dislocation density using electrical resistivity.
4. Low thermal conductivity: Nickel has a low thermal conductivity, which makes it suitable for studying the effect of temperature on dislocation density.
5. High ductility: Nickel is highly ductile, which allows for the study of plastic deformation and its effect on dislocation density.
